# David Alderman
## Econometrics
## Assignment #4

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

col_headers = ['f1','f2','f3','f4','f5','v1','v2','v3','v4','v5','h_income','choice']
data = pd.read_csv('train_heating_2008.txt',sep='\s+',names = col_headers)
n = len(data)

# Part A

In [2]:
mean_by_variable = data.mean()
std_by_variable = (data.var())**.5
col_names = ['Mean','Std Dev']
mean_std = pd.concat([mean_by_variable,std_by_variable],axis=1)
mean_std.columns = col_names

### Part A - Mean

In [3]:
# Xi matrix below reflects the mean of each Xi, e.g. Xi1, Xi2, Xi3, etc.
# row is the coefficient
# column is j choice
Xi_mean_matrix = np.zeros((6,5))
for i in range(5):
    Xi_mean_matrix[0,i] = mean_std.iloc[i,0]
    Xi_mean_matrix[1,i] = mean_std.iloc[i+5,0]
    if i > 0:
        Xi_mean_matrix[i+1,i] = 1

index = ['Beta 1','Beta 2','Alpha 2','Alpha 3','Alpha 4', 'Alpha 5'] 
col_names = ['1','2','3','4','5']
mean_df = pd.DataFrame(data=Xi_mean_matrix,index=index,columns=col_names)
mean_df.round(decimals=4)

,1,2,3,4,5
Beta 1,3.3147,3.3695,3.5920,5.0332,0.777
Beta 2,0.4739,0.6568,0.3997,0.4739,0.413
Alpha 2,0.0000,1.0000,0.0000,0.0000,0.000
Alpha 3,0.0000,0.0000,1.0000,0.0000,0.000
Alpha 4,0.0000,0.0000,0.0000,1.0000,0.000
Alpha 5,0.0000,0.0000,0.0000,0.0000,1.000


### Part A - Standard Deviation

In [4]:
# Similar to Xi mean matrix above, Xi std dev matrix here relfects the standard deviation of each Xi, e.g. Xi1, Xi2, Xi3, etc.
# row is the coefficient
# column is j choice
Xi_std_matrix = np.zeros((6,5))
for i in range(5):
    Xi_std_matrix[0,i] = mean_std.iloc[i,1]
    Xi_std_matrix[1,i] = mean_std.iloc[i+5,1]
    
std_df = pd.DataFrame(data=Xi_std_matrix,index=index,columns=col_names)
std_df.round(decimals=4)

,1,2,3,4,5
Beta 1,0.3472,0.3356,0.3416,0.5008,0.1177
Beta 2,0.0462,0.0693,0.0396,0.0462,0.0523
Alpha 2,0.0000,0.0000,0.0000,0.0000,0.0000
Alpha 3,0.0000,0.0000,0.0000,0.0000,0.0000
Alpha 4,0.0000,0.0000,0.0000,0.0000,0.0000
Alpha 5,0.0000,0.0000,0.0000,0.0000,0.0000


# Part B

### Applying equation below with given gamma vector

$$
L\left ( \gamma  \right ) = \prod_{i} \prod_{j}(\frac{exp(X_{ij}' \gamma)}{\sum_{j} \exp(X_{ij}' \gamma)})^{1_{Y_i = j}}
$$

In [5]:
gamma = np.array([0.1,0.2,0.3,0.4,0.5,0.4]).T

j = 6

choice = np.array(data['choice'])
X1 = np.matrix([np.array(data['f1']),np.array(data['v1']),np.zeros(n),np.zeros(n),np.zeros(n),np.zeros(n)]).T
X2 = np.matrix([np.array(data['f2']),np.array(data['v2']),np.ones(n),np.zeros(n),np.zeros(n),np.zeros(n)]).T
X3 = np.matrix([np.array(data['f3']),np.array(data['v3']),np.zeros(n),np.ones(n),np.zeros(n),np.zeros(n)]).T
X4 = np.matrix([np.array(data['f4']),np.array(data['v4']),np.zeros(n),np.zeros(n),np.ones(n),np.zeros(n)]).T
X5 = np.matrix([np.array(data['f5']),np.array(data['v5']),np.zeros(n),np.zeros(n),np.zeros(n),np.ones(n)]).T

def log_likelihood(gamma):
    output = 1

    for i in range(n):
        X = np.array([X1[i,:],X2[i,:],X3[i,:],X4[i,:],X5[i,:]])
        exp_values = np.exp(np.array(X.dot(gamma)))
        numerator = exp_values[int(choice[i])-1]
        denominator = np.sum(exp_values)
        output = output * numerator/denominator
    output = math.log(output)

    return output

print(np.round(log_likelihood(gamma),4))

-453.4357


# Part E

#### Please note, this is Part E, not Part C. I had issues doing part E below C and D in my program. I believe this is an issue some copying of variables. Thank you for understanding.   

##### Note, Part F is after Part D. Apologies for any confusion

#### To compute the gamma hat vector, we minimize the negative log likelihood function. Effectively changing the defined function in Part A. Part A would output 453.43

#### I use the scipy minimize function to do this. 

In [6]:
from scipy.optimize import minimize
def neg_log_like(gamma):
    likelihood = 1

    for i in range(n):
        X = np.array([X1[i,:],X2[i,:],X3[i,:],X4[i,:],X5[i,:]])
        exp_values = np.exp(np.array(X.dot(gamma)))
        numerator = exp_values[int(choice[i])-1]
        denominator = np.sum(exp_values)
        likelihood = likelihood * numerator/denominator
    log_like = -math.log(likelihood)

    return log_like

#### Gamma vector estimation

In [7]:
model = minimize(neg_log_like,gamma)

gamma_hat = model.x
gamma_hat = np.array(gamma_hat)
print('Gamma Hat Vector:')
print(np.round(gamma_hat,4))

Gamma Hat Vector:
[ -2.6436 -21.8123  -0.5642  -3.1185   2.1758 -12.1638]


#### Standard error of estimation

In [8]:
hess_inv = model.hess_inv
std_err = np.diag(hess_inv)**0.5
print('Standard Error:')
print(np.round(std_err,4))

Standard Error:
[0.5237 4.0412 0.6794 0.4381 0.7984 1.562 ]


# Part C

#### Taking the derivative of the log likelihood function, we derive the first derivative equation below:

$$
\frac{\partial L\left ( \gamma  \right )}{\partial \gamma} = \sum_{i}\sum_{j}1_{Y_{i} = j}\left ( {X_{ij}' - \frac{\sum_{ij}X_{ij}exp\left ( X_{ij}'\gamma  \right )}{\sum_{ij}exp\left ( X_{ij}'\gamma  \right )}} \right )\
$$

### Applying above with the given data and gamma vector

In [9]:
der_matrix = []
for m in range (j):
    derivative = 0
    for i in range(len(data)):
        X = np.zeros((j-1,j))
        for k in range(j-1):
            X[k][0] = data.iloc[i,k]
            X[k][1] = data.iloc[i,k+5]
            if k > 0:
                X[k, k + 1] = 1
        exp_values = np.exp(X.dot(gamma))
        numerator = np.sum(X[:,m].dot(exp_values))
        denominator = np.sum(exp_values)
        derivative += X[int(choice[i])-1,m] - numerator / denominator

    der_matrix.append(derivative)

print(np.round(der_matrix,4))

[-16.9934  -5.7331 -45.5277 -27.1614 -44.8915 -32.2401]


# Part D

#### Taking the second derivative of the log likelihood function, we derive the second derivative equation below:

$$
\frac{\partial^2 L\left ( \gamma  \right )}{\partial \gamma^2} = -\sum_{i} \sum_{j}\frac{(\sum_{j} X_{ij}' X_{ij}exp(X_{ij}' \gamma)(\sum_{j}X_{ij}' \gamma))  - (\sum_j X_{ij}exp(X_{ij}' \gamma)(\sum_j X_{ij}'exp(X_{ij}' \gamma) }{\sum_{j} \exp(X_{ij}' \gamma)} * 1_{Y_i = j}
$$

##### Using loop to numerically solve above equation

In [10]:
der2_matrix = np.zeros((j,j))
for m in range (j):
    for n in range(j):
        der2 = 0
        for i in range(len(data)):
            X = np.zeros((j-1,j))
            for k in range(j-1):
                X[k][0] = data.iloc[i,k]
                X[k][1] = data.iloc[i,k+5]
                if k > 0:
                    X[k, k + 1] = 1
            exp_values = np.exp(X.dot(gamma))
            num1,num2,num3,num4 = 0,0,0,0
            for k in range(j-1):
                num1 += X[k,m] * X[k,n] * exp_values[k]
                num2 += exp_values[k]
                num3 += X[k,m] * exp_values[k]
                num4 += X[k,n] * exp_values[k]
            sec_num = num1 * num2 - num3 * num4
            sec_de = np.square(np.sum(exp_values))
            der2 += -sec_num / sec_de

        der2_matrix[m,n] = der2

print(np.round(der2_matrix,4))

[[-472.9768   -5.1376    4.1741   -7.3609 -109.1099  107.4148]
 [  -5.1376   -2.3511   -8.5342    4.5238    0.7522    2.8764]
 [   4.1741   -8.5342  -39.5939   10.6603   13.8109    8.0683]
 [  -7.3609    4.5238   10.6603  -41.7168   14.8238    8.6606]
 [-109.1099    0.7522   13.8109   14.8238  -49.6633   11.2184]
 [ 107.4148    2.8764    8.0683    8.6606   11.2184  -33.6781]]


# Part F

#### Calculating P3 market share for current data and for a 10% reduction in installation costs

#### Applying equation below

$$
\hat{p}_3 = \frac{1}{N} \sum_i\frac{exp(X_{i3}' \hat{\gamma})}{\sum_jexp(X_{ij}'\hat{\gamma})}
$$

In [11]:
# Defining function to calculate p hat
def prob3(X3,gamma,n):
    p3 = 0
    choice = 3
    for i in range(n):
        X = np.array([X1[i, :], X2[i, :], X3[i, :], X4[i, :], X5[i, :]])
        exp_values = np.exp(np.array(X.dot(gamma)))
        numerator = exp_values[choice-1]
        denominator = np.sum(exp_values)
        p3 += numerator / denominator
    p3 = p3 / n
    return p3

In [12]:
# Given values and creating new installation costs columns for X3 matrix defined above
n = len(data)
IC3 = X3[:,0]
IC3_reduction = 0.10
IC3_new = IC3 * (1-IC3_reduction)
X3_new = X3.copy()
X3_new[:,0] = IC3_new

In [13]:
# Evalating p 3 for original and new installation costs
p3_original = prob3(X3,gamma_hat,n)
print('P3 at original:')
p3_original = p3_original * 100
print('%0.3f%%' % p3_original)

p3_new = prob3(X3_new,gamma_hat,n)
print('\n''P3 at reduced:')
p3_new = p3_new * 100
print('%0.3f%%' % p3_new)

P3 at original:
10.526%

P3 at reduced:
21.818%


In [14]:
# Calculating change
delta = p3_new - p3_original
delta.astype(np.float)
print('Market Share change of when \nreducing installation costs by 10%:')
print('%0.3f%%' % delta)

Market Share change of when 
reducing installation costs by 10%:
11.291%
